In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [79]:
# this allows me to see more rows so I can copy and paste df data
pd.set_option('display.max_rows', 500) 
pd.set_option('display.max_columns', 500) 

In [3]:
df = pd.read_csv(r'C:\DA12\Projects\capstone_refugees_da12\data\population_totals.csv', skiprows=14)

In [5]:
df.head()

,Year,Country of origin,Country of origin (ISO),Country of asylum,Country of asylum (ISO),Refugees under UNHCR's mandate,Asylum-seekers,IDPs of concern to UNHCR,Other people in need of international protection,Stateless persons,Host Community,Others of concern
0,1951,Unknown,UNK,Australia,AUS,180000,0,0,-,0,0,0
1,1951,Unknown,UNK,Austria,AUT,282000,0,0,-,0,0,0
2,1951,Unknown,UNK,Belgium,BEL,55000,0,0,-,0,0,0
3,1951,Unknown,UNK,Canada,CAN,168511,0,0,-,0,0,0
4,1951,Unknown,UNK,Denmark,DNK,2000,0,0,-,0,0,0


In [7]:
df.pop('Country of origin (ISO)')
df.pop('Country of asylum (ISO)')
df.pop('Asylum-seekers')
df.pop('IDPs of concern to UNHCR')
df.pop('Other people in need of international protection')
df.pop('Stateless persons')
df.pop('Host Community')
df.pop('Others of concern')

0           0
1           0
2           0
3           0
4           0
         ... 
125804    133
125805      0
125806      0
125807      0
125808     10
Name: Others of concern, Length: 125809, dtype: int64

In [13]:
df.columns = ['Year', 'Country of Origin', 'Country of Asylum', 'Refugees']

In [15]:
df.head(10)

,Year,Country of Origin,Country of Asylum,Refugees
0,1951,Unknown,Australia,180000
1,1951,Unknown,Austria,282000
2,1951,Unknown,Belgium,55000
3,1951,Unknown,Canada,168511
4,1951,Unknown,Denmark,2000
5,1951,Unknown,France,290000
6,1951,Unknown,United Kingdom of Great Britain and Northern I...,208000
7,1951,Unknown,Germany,265000
8,1951,Unknown,Greece,18000
9,1951,Unknown,"China, Hong Kong SAR",30000


In [21]:
origin = (
    df.groupby('Country of Origin')["Refugees"].sum()
    .reset_index()
    #.sort_values("Refugees", ascending=False)
    .iloc[0:500] 
)
origin

,Country of Origin,Refugees
0,Afghanistan,154402630
1,Albania,417113
2,Algeria,211409
3,Andorra,92
4,Angola,17775243
...,...,...
209,Viet Nam,16620506
210,Western Sahara,6339725
211,Yemen,689146
212,Zambia,59531


In [23]:
asylum = (
    df.groupby('Country of Asylum')["Refugees"].sum()
    .reset_index()
    #.sort_values("Refugees", ascending=False)
    .iloc[0:500] 
)
asylum

,Country of Asylum,Refugees
0,Afghanistan,1256841
1,Albania,55194
2,Algeria,6320758
3,Angola,1855578
4,Anguilla,0
...,...,...
188,Venezuela (Bolivarian Republic of),2558924
189,Viet Nam,912012
190,Yemen,4681043
191,Zambia,5351343


In [27]:
merged = pd.merge(
    left = origin,
    right = asylum,
    left_on='Country of Origin', 
    right_on='Country of Asylum',
    suffixes = ['_origin', '_asylum']
)

In [31]:
merged.head(2)

,Country of Origin,Refugees_origin,Country of Asylum,Refugees_asylum
0,Afghanistan,154402630,Afghanistan,1256841
1,Albania,417113,Albania,55194


In [33]:
merged.pop('Country of Asylum')

0                             Afghanistan
1                                 Albania
2                                 Algeria
3                                  Angola
4                                Anguilla
                      ...                
186    Venezuela (Bolivarian Republic of)
187                              Viet Nam
188                                 Yemen
189                                Zambia
190                              Zimbabwe
Name: Country of Asylum, Length: 191, dtype: object

In [35]:
merged.columns = ['Country', 'count origin', 'count asylum']

In [37]:
merged.head(10)

,Country,count origin,count asylum
0,Afghanistan,154402630,1256841
1,Albania,417113,55194
2,Algeria,211409,6320758
3,Angola,17775243,1855578
4,Anguilla,5,0
5,Antigua and Barbuda,1142,20
6,Argentina,15795,721231
7,Armenia,1917527,4272421
8,Aruba,0,0
9,Australia,573,4566275


In [73]:
# add friendliness column
merged['friendliness'] = round(merged['count asylum'] / (merged['count origin'] + merged['count asylum']) * 100, 2)

In [81]:
merged.sort_values('friendliness', ascending=False)

,Country,count origin,count asylum,friendliness
112,Micronesia (Federated States of),0,10,100.00
164,Sweden,674,6871722,99.99
64,France,2289,15889220,99.99
128,Norway,144,1778322,99.99
9,Australia,573,4566275,99.99
165,Switzerland,784,3529337,99.98
39,"China, Macao SAR",150,672761,99.98
63,Finland,112,554900,99.98
69,Germany,10025,40463645,99.98
51,Denmark,325,1688323,99.98


In [83]:
# who is coming to the US? 

In [85]:
df.head(10)

,Year,Country of Origin,Country of Asylum,Refugees
0,1951,Unknown,Australia,180000
1,1951,Unknown,Austria,282000
2,1951,Unknown,Belgium,55000
3,1951,Unknown,Canada,168511
4,1951,Unknown,Denmark,2000
5,1951,Unknown,France,290000
6,1951,Unknown,United Kingdom of Great Britain and Northern I...,208000
7,1951,Unknown,Germany,265000
8,1951,Unknown,Greece,18000
9,1951,Unknown,"China, Hong Kong SAR",30000


In [89]:
usa = df[df['Country of Asylum'] == 'United States of America']

In [103]:
usa_group = usa.groupby(['Year', 'Country of Origin'])['Refugees'].sum().reset_index()

In [111]:
# Create pivot table with country as rows and year as columns
usa_pivot = usa_group.pivot(index='Year', columns='Country of Origin', values='Refugees')
#usa_pivot - this is what I copied over to excel

In [137]:
usa_refugees = usa.groupby('Country of Origin')['Refugees'].sum().reset_index().sort_values('Refugees', ascending=False)

In [139]:
usa_refugees

,Country of Origin,Refugees
193,Unknown,19831597
151,Russian Federation,2072975
198,Viet Nam,2035489
38,China,1290682
23,Bosnia and Herzegovina,841624
102,Lao People's Dem. Rep.,524603
89,Iran (Islamic Rep. of),469910
167,Somalia,443384
82,Haiti,436157
64,Ethiopia,408477
